In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

In [2]:
def determine_salary(salary_string):
    '''
    На вход подаётся строка зарплаты вида:
    "от 10 до 50"
    "от 10"
    "до 50"
    "по договоренности"
    '''
    
    # salary =re.split(r'[^\d+\s+]+', salary_string)
    answer = {'salary_from': None, 'salary_to': None}

    # определим числа, которые содержатся в строке зарплаты
    salary = re.findall(r'[0-9\s]+', salary_string)
    # уберём html-неразрывный пробел между разрядами числа
    salary = [s.replace('\xa0', '') for s in salary]

    '''
    Если в salary 2 элемента, то в качестве начала и окончания вилки берем salary[0] и salary[1].
    Если в salary 1 элемент, то если строка salary_string начинается с 'от',
    то в качестве начала вилки используем salary[0].
    А если в строке salary_string присутствует слово 'до',
    то в качестве окончания вилки используем salary[0].
    Если же salary не содержит числовых элементов, то записываем None в начало и конец вилки
    '''
    if len(salary) == 2:
        answer['salary_from'] = int(salary[0])
        answer['salary_to'] = int(salary[1])
    elif salary_string.split()[0] == 'от':
        answer['salary_from'] = int(salary[0])
    elif salary_string.split().count('до') == 1:
        answer['salary_to'] = int(salary[0])

    return answer

In [3]:
def clear_link(link):
    link = link[:link.find('//')]+'//'+link[link.find('hh.ru'):link.find('?')]
    return link

In [4]:
def get_hh_data(keyword, link):
    link_origin = 'https://www.hh.ru'
    headers = {'Accept': '*/*',
               'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0'}
    html = requests.get(link, headers=headers).text
    parsed_html = bs(html, 'html.parser')
    
    vacancy_list = parsed_html.findAll('div', {'data-qa': 'vacancy-serp__vacancy'})

    #определим ссылку на следующую страницу. Это ссылка на кнопку "Далее"
    next_link = link_origin + parsed_html.find('a',{'data-qa':'pager-next'})['href']
    
    #создадим списки с данными, которые будем использовать в качестве столбцов датафрейма
    list_vacancy_link = []
    list_vacancy_text = []
    list_vacancy_company = []
    list_vacancy_place = []
    list_salary_from = []
    list_salary_to = []

    for vacancy in vacancy_list:
        #подчистим ссылку на вакансию
        vacancy_link = clear_link(vacancy.find('a',{'class':'bloko-link'})['href'])
        
        vacancy_text = vacancy.find('a',{'class':'bloko-link'}).getText()
        vacancy_firm = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-employer'}).getText()
        vacancy_place = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-address'}).getText()
        try:
            salary_string = vacancy.find('div', {'class': 'vacancy-serp-item__compensation'}).text
            salary_dict = determine_salary(salary_string)
        except:
            salary_dict = {'salary_from':None, 'salary_to':None}
        
        #добавим полученные данные в списки
        list_vacancy_link.append(vacancy_link)
        list_vacancy_text.append(vacancy_text)
        list_vacancy_company.append(vacancy_firm)
        list_vacancy_place.append(vacancy_place)
        list_salary_from.append(salary_dict['salary_from'])
        list_salary_to.append(salary_dict['salary_to'])      
    
    #загрузим списки в датафрейм
    vacancy_info = pd.DataFrame({'vacancy_link':list_vacancy_link,
                             'vacancy_text':list_vacancy_text,
                             'vacancy_company':list_vacancy_company,
                             'vacancy_place':list_vacancy_place,
                             'salary_from':list_salary_from,
                             'salary_to':list_salary_to})
    return_data = {'data':vacancy_info, 'next_link':next_link}
    return return_data
   

In [5]:
keyword = input('Введите ключевое слово, по которому будет выполняться поиск подходящих вакансий:  ')
n_pages = int(input('Введите количество страниц поиска:  '))

#основной датафрейм vacancy_list - к нему будем присоединять результаты обхода страниц
vacancy_list = pd.DataFrame({'vacancy_link':[],
                             'vacancy_text':[],
                             'vacancy_company':[],
                             'vacancy_place':[],
                             'salary_from':[],
                             'salary_to':[]})
#ссылка на первую страницу
#Пришлось использовать параметр geo для обхода автоподстановки местоположения
link_hh = link = f'https://hh.ru/search/vacancy?area=1&st=searchVacancy&text={keyword}'
page = 1
while page <= n_pages:
    
    
    data = get_hh_data(keyword, link_hh)
    vacancy_info = data['data']
    
    #на следующем шаге цикла вместо первоначальной ссылки используем ссылку на следующую страницу
    link_hh = data['next_link']
    
    #добавляем полученные данные к vacancy_list
    vacancy_list = pd.concat([vacancy_list,vacancy_info], axis=0, ignore_index=True)
    page +=1
    
vacancy_list['website'] = 'www.hh.ru'

vacancy_list

Введите ключевое слово, по которому будет выполняться поиск подходящих вакансий:  Программист
Введите количество страниц поиска:  5


,vacancy_link,vacancy_text,vacancy_company,vacancy_place,salary_from,salary_to,website
0,https://hh.ru/vacancy/33814598,Программист 1 С,Энергосистемы и Технологии,"Москва, Люблино и еще 1",120000.0,140000.0,www.hh.ru
1,https://hh.ru/vacancy/33940776,.NET C# Разработчик / .NET C# Developer Norway,ООО PaleBlue,Москва,200000.0,300000.0,www.hh.ru
2,https://hh.ru/vacancy/33812932,Программист в Германию,ИП Сметанин Владимир Анатольевич,Москва,2800.0,3800.0,www.hh.ru
3,https://hh.ru/vacancy/32697772,Инженер-программист ( front-end developer ) ju...,Институт прикладных экономических исследовани...,"Москва, Юго-Западная",100000.0,140000.0,www.hh.ru
4,https://hh.ru/vacancy/33897746,ГИС программист (front),РеИнформ Инт.,Москва,140000.0,NaN,www.hh.ru
5,https://hh.ru/vacancy/33483197,Программист С++,ЮРИОН,Москва,90000.0,110000.0,www.hh.ru
6,https://hh.ru/vacancy/34099140,Backend разработчик (.NET),"АйТи Гео, ООО (IT GEO)","Москва, Бауманская",150000.0,170000.0,www.hh.ru
7,https://hh.ru/vacancy/33897090,ГИС программист (back),РеИнформ Инт.,Москва,140000.0,NaN,www.hh.ru
8,https://hh.ru/vacancy/34169648,Программист 1С8,Вартон,"Москва, Кунцевская",100000.0,NaN,www.hh.ru
9,https://hh.ru/vacancy/34202529,Front-end разработчик (Senior),Foundook,Москва,NaN,250000.0,www.hh.ru
